In [1]:
### This script tries to establish baseline for longformer
### in this folder, we will use LegalBERT to run a MLM on 
### contractnli and see what score it gets
### what metric dees LegalBERT paper use? the training loss, so default huggingface loss should be used

In [2]:
### get the contractnli dataset

### load contractnli

from datasets import load_dataset, DatasetDict, Dataset
import json
from transformers import PerceiverTokenizer, PerceiverModel, PerceiverConfig, PerceiverPreTrainedModel, PerceiverForSequenceClassification, TrainingArguments, Trainer, \
    DataCollatorWithPadding, AutoTokenizer, AutoModelForMaskedLM, DataCollatorForLanguageModeling


import re
import os
from tqdm import tqdm
import torch

ROOT_PATH = "/home/yan_xu_uk_qbe_com/scc_yan/"

with open(os.path.join(ROOT_PATH, "ignored_dir/data/contract-nli/train.json")) as train_json_f:
    train_json = json.load(train_json_f)

id2label = {0: "Entailment", 1: "Contradiction", 2: "NotMnetioned"}
label2id = {"Entailment": 0, "Contradiction": 1, "NotMentioned": 2}

def load_dataset_custom(dataset_name):
    if dataset_name == "contract-nli":
        def contract_nli_iterator(data):
            documents, labels = data['documents'], data['labels']
            for document in documents:
                id = document['id']
                file_name = document['file_name']
                text = document['text']
                spans = document['spans']
                annotation_sets = document['annotation_sets']
                document_type = document['document_type']
                url = document['url']
                for annotation_id, annotation_content in annotation_sets[0]['annotations'].items():
                    hypothesis = labels[annotation_id]['hypothesis']
                    choice = annotation_content['choice']
                    yield {
                        "id": id,
                        "file_name": file_name,
                        "text": text,
                        "spans": spans,
                        "document_type": document_type,
                        "url": url,
                        "hypothesis": hypothesis,
                        "labels": label2id[choice],
                    }            
        base_filepath = os.path.join(ROOT_PATH, "ignored_dir/data/contract-nli")
        train_filepath = os.path.join(base_filepath, "train.json")
        validation_filepath = os.path.join(base_filepath, "dev.json")
        test_filepath = os.path.join(base_filepath, "test.json")
        with open(train_filepath) as f:
            train_data = json.load(f)
        with open(validation_filepath) as f:
            validation_data = json.load(f)
        with open(test_filepath) as f:
            test_data = json.load(f)
        data = {
            "train": Dataset.from_generator(lambda: contract_nli_iterator(train_data)),
            "validation": Dataset.from_generator(lambda: contract_nli_iterator(validation_data)),
            "test": Dataset.from_generator(lambda: contract_nli_iterator(test_data)),
        }
        return DatasetDict(data)
    return None

contractnli_dataset = load_dataset_custom("contract-nli")

In [3]:
### get tokenizer and model for LegalBERT

legalbert_tokenizer = AutoTokenizer.from_pretrained("nlpaueb/legal-bert-base-uncased")

/home/yan_xu_uk_qbe_com/scc_yan/virtual-env/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [4]:
### chunk contractnli dataset into sizes of 512

def chunk_contractnli(ds, cs, os):
    sep_token = legalbert_tokenizer.sep_token
    concat_text = sep_token.join([sep_token.join([e['text'], e['hypothesis']]) for e in ds])
    tokenized_text = legalbert_tokenizer(concat_text).input_ids
    i = 0
    chunks = []
    with tqdm(total=100) as pbar:
        while i < len(tokenized_text):
            if i + cs >= len(tokenized_text):
                chunks.append(tokenized_text[i:])
                i = len(tokenized_text)
            else:
                chunks.append(tokenized_text[i: i + cs])
                i += (cs - os)
            pbar.update(i / len(tokenized_text))
    return Dataset.from_dict({'input_ids': chunks})

cs = 512
os = 50
contractnli_chunked = DatasetDict({mode: chunk_contractnli(contractnli_dataset[mode], cs, os) for mode in ['train', 'validation', 'test']})

Token indices sequence length is longer than the specified maximum sequence length for this model (15926735 > 512). Running this sequence through the model will result in indexing errors
17237.77380593072it [00:00, 38131.65it/s] 
2792.178550865565it [00:00, 40724.00it/s]
5100.524889456003it [00:00, 40221.16it/s] 


In [5]:
# not using this for now
# trainer_args = TrainingArguments(output_dir="baseline_dir", deepspeed="ds_config.json")

In [6]:
legalbert_model = AutoModelForMaskedLM.from_pretrained("nlpaueb/legal-bert-base-uncased")

### legalbert_tokenizer.model_max_length shows 512

In [7]:
data_collator = DataCollatorForLanguageModeling(tokenizer=legalbert_tokenizer)

trainer = Trainer(model=legalbert_model, eval_dataset=contractnli_chunked['test'], data_collator=data_collator)
# trainer = Trainer(model=legalbert_model, args=trainer_args, eval_dataset=contractnli_chunked['test'], data_collator=data_collator)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [8]:
trainer.evaluate()

### on validation set
### eval_loss: 0.5897765159606934
## with deepspeed eval_loss gives 0.5909032821656273

### on test set
### without deepspeed: 
# {'eval_loss': 0.6284850239753723,
# 'eval_runtime': 97.6967,
# 'eval_samples_per_second': 104.405,
# 'eval_steps_per_second': 1.638}
### with deepspeed: 0.6288514733314514,

[2024-08-16 13:17:45,453] [INFO] [real_accelerator.py:203:get_accelerator] Setting ds_accelerator to cuda (auto detect)
 [WARNING]  async_io requires the dev libaio .so object and headers but these were not found.
 [WARNING]  async_io: please install the libaio-dev package with apt
 [WARNING]  If libaio is already installed (perhaps from source), try setting the CFLAGS and LDFLAGS environment variables to where it can be found.
 [WARNING]  Please specify the CUTLASS repo directory as environment variable $CUTLASS_PATH


/opt/conda/compiler_compat/ld: cannot find -laio: No such file or directory
collect2: error: ld returned 1 exit status


 [WARNING]  sparse_attn requires a torch version >= 1.5 and < 2.0 but detected 2.4
 [WARNING]  using untested triton version (3.0.0), only 1.0.0 is known to be compatible


/home/yan_xu_uk_qbe_com/scc_yan/virtual-env/lib/python3.10/site-packages/deepspeed/runtime/zero/linear.py:49: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  def forward(ctx, input, weight, bias=None):
/home/yan_xu_uk_qbe_com/scc_yan/virtual-env/lib/python3.10/site-packages/deepspeed/runtime/zero/linear.py:67: FutureWarning: `torch.cuda.amp.custom_bwd(args...)` is deprecated. Please use `torch.amp.custom_bwd(args..., device_type='cuda')` instead.
  def backward(ctx, grad_output):
/home/yan_xu_uk_qbe_com/scc_yan/virtual-env/lib/python3.10/site-packages/torch/nn/parallel/parallel_apply.py:79: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.device(device), torch.cuda.stream(stream), autocast(enabled=autocast_enabled):
/home/yan_xu_uk_qbe_com/scc_yan/virtual-env/lib/python3.10/site-packages/torch/nn/paralle

{'eval_loss': 0.6284850239753723,
 'eval_runtime': 99.3761,
 'eval_samples_per_second': 102.64,
 'eval_steps_per_second': 1.61}